# Optimizing our Models

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

## Load and Process the Data
This is using the same procedure as in housing_prices.ipynb. It would make sense to extract this into its own distinct process so that we didn't have to duplicate the code.

In [2]:
df = pd.read_csv('data/housing.csv')
df = df.dropna()

In [3]:
X = df.drop('median_house_value', axis=1)
y = df['median_house_value']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42)

In [5]:
# Define the categories in the desired order for encoding
categories = [['INLAND', '<1H OCEAN', 'NEAR OCEAN', 'NEAR BAY', 'ISLAND']]
encoder = OrdinalEncoder(categories=categories)
X_train['ocean_proximity'] = encoder.fit_transform(
    X_train[['ocean_proximity']])

In [6]:
X_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
19566,-120.96,37.61,23.0,3497.0,887.0,2467.0,816.0,1.9444,0.0
7292,-118.22,33.98,34.0,2225.0,753.0,2980.0,736.0,1.6685,1.0
17618,-121.94,37.28,27.0,2859.0,464.0,1144.0,430.0,5.0822,1.0
17518,-121.91,37.34,35.0,2189.0,607.0,1193.0,562.0,2.8042,1.0
5172,-118.28,33.95,41.0,835.0,208.0,707.0,192.0,1.4103,1.0


In [7]:
y_train.head()

19566     93400.0
7292     128800.0
17618    327500.0
17518    240900.0
5172      86200.0
Name: median_house_value, dtype: float64

Encode the ocean_proximity column in our validation and test sets as well.

In [8]:
X_test['ocean_proximity'] = encoder.fit_transform(
    X_test[['ocean_proximity']])

## Optimize our Models with Grid Search
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

The function below performs a grid search. To use it, we will define the parameters (including the type of model we wish to optimize).

In [9]:
def perform_grid_search(parameters):
    results = {}
    for name, setup in parameters.items():
        grid_search = GridSearchCV(
            setup['model'],
            setup['params'],
            cv=3,
            scoring='neg_mean_squared_error',
            n_jobs=-1)

        grid_search.fit(X_train, y_train)

        results[name] = {
            'best_score': grid_search.best_score_,
            'best_params': grid_search.best_params_}
    return results

### Sidebar: k-Fold Cross-Validation
You may have noticed the cv=3 argument to GridSearchCV. This refers to "cross-validation".

k-Fold Cross-Validation is a technique used to evaluate how well a machine learning model will perform on unseen data. Instead of splitting the data into just one training set and one test set, k-fold cross-validation splits the data into k equal-sized parts called folds.

1. The dataset is shuffled (optional, but usually recommended) to make sure the data is randomly ordered.
2. The data is then split into k folds (for example, if k=3, you split the data into 3 parts of roughly equal size).
3. The model is trained k times — each time using k-1 folds for training and the remaining fold for validation.
4. Each fold gets a turn to be the validation set exactly once.
5. At the end, you calculate the average performance across all k runs to get a more reliable estimate of how the model performs.


#### Example (k=3)

| Fold # | Training Folds | Validation Fold |
|---|---|---|
| 1 | Folds 2, 3 | Fold 1 |
| 2 | Folds 1, 3 | Fold 2 |
| 3 | Folds 1, 2 | Fold 3 |



#### Why Use k-Fold Cross-Validation?

- It makes better use of your data — instead of losing data to a fixed validation set, every data point gets used for both training and validation.
- It gives a more reliable estimate of performance, because you're testing on multiple different slices of the data.
- It helps you detect overfitting — if the model does great on some folds but poorly on others, that's a sign the model might be too sensitive to the training data.

#### k-Fold for Regression vs Classification

- For regression, you can split the data into k regular folds (each fold gets a roughly equal number of samples).
- For classification, you might want to use Stratified k-Fold. This ensures that each fold contains roughly the same proportion of each class (important if your classes are imbalanced).

### Decision Tree

In [10]:
decision_tree_params = {
    'DecisionTree': {
        'model': DecisionTreeRegressor(random_state=42),
        'params': {
            'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
            #'splitter': ['best', 'random'],
            'max_depth': [None, 10],
            'min_samples_split': [2, 3],
            'min_samples_leaf': [1, 14, 20],
            'min_weight_fraction_leaf': [0.0, 0.01],
            'max_features': [None, 'sqrt', 'log2'],
            'max_leaf_nodes': [None, 100, 150],
            'min_impurity_decrease': [0.0, 0.01],
            'ccp_alpha': [0.0, 0.01, 0.02]
        }
    }
}

In [11]:
perform_grid_search(decision_tree_params)

{'DecisionTree': {'best_score': -3427705884.6115246,
  'best_params': {'ccp_alpha': 0.0,
   'criterion': 'poisson',
   'max_depth': None,
   'max_features': None,
   'max_leaf_nodes': None,
   'min_impurity_decrease': 0.01,
   'min_samples_leaf': 14,
   'min_samples_split': 2,
   'min_weight_fraction_leaf': 0.0}}}

### Random Forest

In [12]:
random_forest_params = {
    'RandomForest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {
            'n_estimators': [10, 100, 400],
            'max_features': [None, 'sqrt', 'log2']
        }
    }
}

In [13]:
perform_grid_search(random_forest_params)

{'RandomForest': {'best_score': -2573219444.0531,
  'best_params': {'max_features': 'sqrt', 'n_estimators': 400}}}

### Gradient Boosting

In [14]:
gradient_boosting_params = {
    'HistGradientBoosting': {
        'model': HistGradientBoostingRegressor(random_state=42),
        'params': {
            'max_iter': [50, 100, 150],
            'learning_rate': [0.01, 0.1, 0.2]
        }
    }
}

In [15]:
perform_grid_search(gradient_boosting_params)

{'HistGradientBoosting': {'best_score': -2300070904.601216,
  'best_params': {'learning_rate': 0.2, 'max_iter': 150}}}

### XGBoost

In [16]:
xgboost_params = {
    'XGBoost': {
        'model': xgb.XGBRegressor(random_state=42),
        'params': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 6, 9],
            'subsample': [0.5, 0.7, 1.0],
            'colsample_bytree': [0.5, 0.7, 1.0]
        }
    }
}

In [17]:
perform_grid_search(xgboost_params)

{'XGBoost': {'best_score': -2265082406.696934,
  'best_params': {'colsample_bytree': 1.0,
   'learning_rate': 0.1,
   'max_depth': 6,
   'n_estimators': 300,
   'subsample': 0.7}}}

## Build and Evaluate Models Using Optimized Hyperparameters

In [18]:
from sklearn.metrics import mean_squared_error, r2_score

### Decision tree

Best parameters from grid search:
'ccp_alpha': 0.0, 'criterion': 'poisson', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.01, 'min_samples_leaf': 14, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0

In [19]:
tree_regressor = DecisionTreeRegressor(
    ccp_alpha=0.0, 
    criterion='poisson',
    max_depth=None,
    max_features=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.01,
    min_samples_leaf=14,
    min_samples_split=2,
    min_weight_fraction_leaf=0.0,
    random_state=42)

tree_regressor.fit(X_train, y_train)

y_pred_tree = tree_regressor.predict(X_test)

mse_tree = mean_squared_error(y_test, y_pred_tree)
r2_tree = r2_score(y_test, y_pred_tree)

print("Decision Tree MSE:", mse_tree)
print("Decision Treet R² Score:", r2_tree)

Decision Tree MSE: 3152051207.5525823
Decision Treet R² Score: 0.7635535831938253


 ### Random forest
 Best parameters from grid search: 'max_features': 'sqrt', 'n_estimators': 400

In [20]:
forest_regressor = RandomForestRegressor(
    max_features='sqrt',
    n_estimators=400,
    random_state=42)

forest_regressor.fit(X_train, y_train)

y_pred_forest = forest_regressor.predict(X_test)

mse_forest = mean_squared_error(y_test, y_pred_forest)
r2_forest = r2_score(y_test, y_pred_forest)

print("Random Forest MSE:", mse_forest)
print("Random Forest R² Score:", r2_forest)

Random Forest MSE: 2371179840.1664042
Random Forest R² Score: 0.8221294833450031


### Gradient boosting
Best parameters from grid search: 'learning_rate': 0.2, 'max_iter': 150

In [21]:
hist_regressor = HistGradientBoostingRegressor(
    learning_rate=0.2,
    max_iter=150,
    random_state=42)

hist_regressor.fit(X_train, y_train)

y_pred_hist = hist_regressor.predict(X_test)

mse_hist = mean_squared_error(y_test, y_pred_hist)
r2_hist = r2_score(y_test, y_pred_hist)

print("HistGradientBoostingRegressor MSE:", mse_hist)
print("HistGradientBoostingRegressor R²:", r2_hist)

HistGradientBoostingRegressor MSE: 2165946320.011851
HistGradientBoostingRegressor R²: 0.8375247695423809


### XGBoost
Best parameters from grid search: 'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 300, 'subsample': 0.7

In [22]:
xgb_reg = xgb.XGBRegressor(
    colsample_bytree=1.0,
    learning_rate=0.1,
    max_depth=6,
    n_estimators=300,
    subsample=0.7,
    seed=42)

xgb_reg.fit(X_train, y_train)

y_pred_xgb = xgb_reg.predict(X_test)

mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("XGBoost MSE:", mse_xgb)
print("XGBoost R² Score:", r2_xgb)

XGBoost MSE: 2124919474.2667944
XGBoost R² Score: 0.8406023371421821


# Activities

## Perform a Grid Search
Compare the results of using grid search to tune hyperparameters with the results of the corresponding models in the housing_prices.ipynb notebook. Did we improve on the results for all classifiers?

Where we got worse results, this suggests that we didn't configure our search to explore thoroughly enough. Examine the hyperparameters for each model type and use those to prepare a more thorough hyperparameter optimization.
 - https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
 - https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
 - https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html
 - https://xgboost.readthedocs.io/en/stable/python/python_api.html
 - https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html

Experiment with hyperparameter optimization and compare your results with the default regressors we used in the ensemble methods session (housing_prices.ipynb).

__CAUTION__: The goal is to get familiar with hyperparameter tuning, not to use electricity for hours and hours training models. It is easy to get carried away!

## Activity: Examine Other Solutions
Examine the solutions that other people have developed for the California Housing Prices dataset with a view to understanding how they have tried to improve their models. For example, user OMARAYMANATIA achieves an MSE of 48911 at https://www.kaggle.com/code/omaraymanatia/california-housing-prices-prediction.

Kaggle conveniently shows you the rating of users (OMARAYMANATIA is a rated Kaggle Expert because of his notebooks) so we can focus on the work of the more experienced Kaggle users.

From your explorations, compile a list of questions and ideas to try. Sort these by priority, taking into account how much work is involved (effort) and estimated likely payoff (value). Low-effort high-value actions are a great place to continue building your knowledge and skills. What list did you come up with?



## Optional Activity: Experiment with Other Optimizers 
Scikit-learn has other hyperparameter optimizers and there are other optimization libraries. You may find that you get better results more quickly with those. For example, the Optuna code below found a set of hyperparameter values in just over 30 seconds that were as good as the ones that took GridSearch over 12 minutes to find. See https://optuna.org/ and for more information.

In [23]:
import optuna  # Optuna needs to be installed first
from sklearn.model_selection import cross_val_score

In [24]:
def objective(trial):
    # For ease of comparison, these were kept similar to values for used GridSearch above.
    # You are welcome to experiment further.
    params = {
        'criterion': trial.suggest_categorical('criterion', ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']),
        'max_depth': trial.suggest_categorical('max_depth', [None, 10]),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 3),
        'min_samples_leaf': trial.suggest_categorical('min_samples_leaf', [1, 14, 20]),
        'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 0.0, 0.01),
        'max_features': trial.suggest_categorical('max_features', [None, 'sqrt', 'log2']),
        'max_leaf_nodes': trial.suggest_categorical('max_leaf_nodes', [None, 100, 150]),
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.01),
        'ccp_alpha': trial.suggest_float('ccp_alpha', 0.0, 0.02)
    }

    # Create a Decision Tree Regressor with parameters to be tuned
    model = DecisionTreeRegressor(random_state=42, **params)
    # Use cross-validation to evaluate the model
    scores = cross_val_score(model, X_train, y_train,
                             cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
    # Since scores are negative mean squared error, we negate it to get positive MSE
    mse = -scores.mean()
    return mse


# Create a study object that will find the hyperparameters that minimize the objective
study = optuna.create_study(direction='minimize')
# You can adjust the number of trials here
study.optimize(objective, n_trials=100)

# Fetch the best parameters and the best score achieved
best_params = study.best_params
best_score = study.best_value

print("Best score:", best_score)
print("Best parameters:", best_params)

[I 2025-06-12 00:35:14,784] A new study created in memory with name: no-name-9ab4c705-81c0-476b-aab1-4bce0fde0ce5
[I 2025-06-12 00:35:16,266] Trial 0 finished with value: 4795047814.47474 and parameters: {'criterion': 'absolute_error', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 14, 'min_weight_fraction_leaf': 0.004079711121604664, 'max_features': 'log2', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.00200948113889907, 'ccp_alpha': 0.017071355912086895}. Best is trial 0 with value: 4795047814.47474.
[I 2025-06-12 00:35:16,313] Trial 1 finished with value: 4388150757.327019 and parameters: {'criterion': 'squared_error', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 20, 'min_weight_fraction_leaf': 0.0012513352173886684, 'max_features': 'sqrt', 'max_leaf_nodes': 150, 'min_impurity_decrease': 0.00925011302077775, 'ccp_alpha': 0.011067456539850558}. Best is trial 1 with value: 4388150757.327019.
[I 2025-06-12 00:35:16,359] Trial 2 finished with value

Best score: 3427705884.6115246
Best parameters: {'criterion': 'poisson', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0014381773497878169, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.009141702213870312, 'ccp_alpha': 0.016096535060756943}
